# Script : tri de dossier

## Étape 1 : définir les librairies

Les librairies nécessaires ont été définies au fur et à mesure de l'avancement de la construction du programme. 

Ici, nous avons utilisé **pathlib** afin de pouvoir se promener dans les dossiers et trouver les fichiers

Ensuite **docx** et **PyPDF2** ont été utile pour lire les documents et **shutil** pour déplacer les documents

Il y a le cas de PDF qui sont des images. Pour cela, il faut procéder à la reconnaissance de ceux-ci. **PyTesseract** lira les images et les convertira en texte, ensuite la librairie **pdf2image** convertira le pdf en images directement avec un outil appelé **poppler**.

Il faut activer **sudo** sur l'ordinateur et installer **poppler** et **tesseract**

In [31]:
from pathlib import Path
from docx import Document
from PyPDF2 import PdfReader
import shutil
import pytesseract 
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

from pdf2image import convert_from_path

POPPLER_PATH = r"C:\Program Files\poppler-25.11.0\Library\bin"  # à adapter


## Étape 2 : Lecture des documents

Nous allons ici déjà définir le dossier, où via deux boucles for et la fonction "glob" du package "Path" dans pathlib, définir la liste de tous les documents pdf et docx présents dans la boucle

In [ ]:


documents = Path(r"") # Dossier contenant les documents en vrac

dossier_annonce = Path(r"") # Dossier où seront déplacés les annonces

dossier_CV = Path(r"") # Dossier où seront déplacés les CV


data=[]
for pattern in ("*.docx", "*.pdf"):
    for file in documents.glob(pattern):
        data.append(file)


# Étape préliminaire à l'étape 3

On va créer une fonction qui traduira les images en texte

In [29]:
def extract_text_from_scanned_pdf(path):
    pages = convert_from_path(path, dpi=300, poppler_path=POPPLER_PATH)
    texte = ""
    for page in pages:
        texte += pytesseract.image_to_string(page, lang="fra") + "\n"
    return texte

## Étape 3 : Lecture du document, détection du type de document

Nous allons créer une première grande boucle pour parcourir toute la liste "data". Comme vu, il y a  dedans des chemins vers des PDF ou vers des docx.
Nous ouvrons ainsi les docx et pdf selon la méthode nécessaire à cela, et nous récupèrons un texte.

Dans ce texte, nous faisons un score de mots clef. Nous allons définir la fréquence de présence de ces mots, selon s'il s'agit d'une annonce ou bien d'un CV. Pour cela j'ai demandé à un LLM les mots les plus fréquents dans les annonces n'étant pas dans les CV et inversement.

J'ai personnalisé en rajoutant mon prénom, nom, entreprise dans lesquelles j'ai travaillé, et aussi vérifié pour les accents

In [ ]:
keywords_cv = [
        "experience", "competence", "formation", "profil", "curriculum",
        "contact", "diplome", "langues", "logiciels"
    ]

keywords_annonce = [
        "mission", "entreprise", "poste", "responsabilites",
        "profil recherche", "profil recherché", "salaire", "candidature",
        "contrat", "recrutement","nous","recherchons" 
    ]

Une fois le score établi, nous allons poser une condition, il faut que le document ait au moins un score de 1 soit dans le nombre de mot de CV présent dans ce document, soit dans l'annonce : j'ai des documents n'étant ni des annonces, ni des CV

Si le score_CV est supérieur à celui du score de l'annonce, alors il sera mis stocké dans le dossier CV, sinon, il sera stocké dans le dossier annonce

In [34]:
for i in range(len(data)):
    
    if data[i].suffix.lower() == ".docx":
        texte="\n".join(p.text for p in Document(data[i]).paragraphs)
    
    if data[i].suffix.lower() == ".pdf":
        with open(data[i], "rb") as f:
            reader = PdfReader(f)
            texte = "\n".join(page.extract_text() or "" for page in reader.pages)
            if texte.strip() =="":
                texte=extract_text_from_scanned_pdf(data[i])
            else:
                continue
    
    score_cv=sum(texte.lower().count(mot) for mot in keywords_cv)
    score_annonce=sum(texte.lower().count(mot) for mot in keywords_annonce)
    print(score_cv, score_annonce)
    if score_cv > 1 or score_annonce > 1:

        if score_cv < score_annonce:
            shutil.move(str(data[i]), str(dossier_annonce))
        
        elif score_cv >= score_annonce:
            shutil.move(str(data[i]), str(dossier_CV))
    
    else:
        continue


5 12
4 13
15 3


# Conclusion

Le code est fonctionnel.

Points d'amélioration : 
 - Il me reste les lettres de motivations :-)
 - Certaines annonces n'ont pas bougé : certaines étant en anglais, d'autres usant d'autres mots, et il se peut qu'il y ait des encodages parfois différent